In [16]:
import torch
import pandas as pd
import time
from tqdm import tqdm

In [2]:
# 1. embedding vector 텐서 파일 불러와서 shape 확인 : torch.Size([12491, 3840])
concatenated_embedding_vectors = torch.load('../data/JobDescription/tensor_data.pth')
concatenated_embedding_vectors.shape

torch.Size([12491, 3840])

In [3]:
# 2. url 가져와서 텐서와 붙이는 함수 정의
def vector_with_url(tensor):
    url = pd.read_csv('../data/JobDescription/pre_result.csv')['출처 URL']
    vector_with_url = {}
    for row in range(len(url)):
        vector_with_url[url[row]] = tensor[row]
    return vector_with_url

In [4]:
# 3. URL 쌍 간의 코사인 유사도를 계산하여 DataFrame을 생성하는 함수 정의
def cos_similarity_df(data):
    url1_list = []
    url2_list = []
    cos_list = []

    # 모든 URL 쌍에 대해 코사인 유사도 계산
    for url1, tensor1 in data.items():
        for url2, tensor2 in data.items():
            if url1 != url2: # 자기 자신과의 계산은 뺍니다!
                url1_list.append(url1)
                url2_list.append(url2)
                cosine_similarity = torch.cosine_similarity(tensor1, tensor2, dim=0)
                cos_list.append(cosine_similarity.item())

    # DataFrame 생성
    df = pd.DataFrame({
        'url1': url1_list,
        'url2': url2_list,
        'cosine-similarity': cos_list
    })

    return df


## 3개 데이터로만 TEST해본 결과 잘 돌아갑니다.

In [ ]:
# 임베딩 벡터에 url을 붙여줍니다.
new_embedding_vectors = vector_with_url(concatenated_embedding_vectors)
new_embedding_vectors

In [6]:
# 이 중 3개 데이터만 선택해서 테스트해보겠습니다.
selected_keys = list(new_embedding_vectors.keys())[:3]
selected_data = {key: new_embedding_vectors[key] for key in selected_keys}
selected_data

{'https://www.wanted.co.kr/wd/167436': tensor([-0.0482, -0.0684,  0.1829,  ..., -0.2828,  0.1753,  0.2709]),
 'https://www.wanted.co.kr/wd/167433': tensor([ 0.0370, -0.0011,  0.1116,  ..., -0.3179,  0.2235,  0.1840]),
 'https://www.wanted.co.kr/wd/167134': tensor([ 0.0258,  0.0194,  0.1455,  ..., -0.2688,  0.2132,  0.1755])}

In [8]:
# DataFrame 생성 및 csv 파일 생성
col_name = '전체컬럼'

df = cos_similarity_df(selected_data)
df.to_csv(f"../data/JobDescription/cosine_cal_{col_name}_test.csv",index=False)

### Tensor를 5등분하여, 하나의 col 혹은 두 개 이상의 col을 선택할 수 있도록 하는 부분입니다.

In [67]:
# 텐서를 5개의 부분으로 나눕니다.
tensor_parts = torch.chunk(concatenated_embedding_vectors, chunks=5, dim=1)

# 각 부분의 크기 출력 : (torch.Size([12491, 768]))인지 확인
for i, part in enumerate(tensor_parts):
    print(f"Part {i} size: {part.size()}")
print("====================")

# 나누어진 텐서 확인
for i, part in enumerate(tensor_parts):
    print(f"Part {i}:")
    print(part)

Part 0 size: torch.Size([12491, 768])
Part 1 size: torch.Size([12491, 768])
Part 2 size: torch.Size([12491, 768])
Part 3 size: torch.Size([12491, 768])
Part 4 size: torch.Size([12491, 768])
Part 0:
tensor([[-0.0482, -0.0684,  0.1829,  ..., -0.2137,  0.1857,  0.1891],
        [ 0.0370, -0.0011,  0.1116,  ..., -0.1033,  0.0084,  0.0612],
        [ 0.0258,  0.0194,  0.1455,  ..., -0.1985,  0.0530,  0.0386],
        ...,
        [ 0.0802, -0.0974,  0.1981,  ..., -0.2107,  0.1324,  0.2148],
        [ 0.1956, -0.1600,  0.2822,  ..., -0.3600,  0.2164,  0.2478],
        [ 0.1956, -0.1600,  0.2822,  ..., -0.3600,  0.2164,  0.2478]])
Part 1:
tensor([[ 0.0473,  0.0635,  0.0188,  ..., -0.1055,  0.0149, -0.0336],
        [-0.1230, -0.1551,  0.1149,  ..., -0.3117,  0.1706,  0.0673],
        [ 0.0788, -0.0896,  0.0954,  ..., -0.3110,  0.1475,  0.1109],
        ...,
        [ 0.2323, -0.0134,  0.1699,  ..., -0.3207,  0.1293,  0.1575],
        [ 0.1135,  0.0599,  0.1207,  ..., -0.1289,  0.0667,  0.1798

## 실제 코드 작업하실 때는 여기 아래부터 수정해주시면 됩니다!

In [9]:
# ['자격요건', '우대조건', '복지', '회사소개', '주요업무'] 하나씩 분리합니다. 차례대로 part 0,1,2,3,4입니다.
# ex) ['자격요건'] 하나를 선택하는 경우 tensor_parts[0]을 입력해주시면 됩니다.
one_tensor = tensor_parts[0]

# one_tensor 사이즈 확인
print(f"Combined Tensor Size: {one_tensor.size()}")
print("Combined Tensor:")
print(one_tensor)

Combined Tensor Size: torch.Size([12491, 768])
Combined Tensor:
tensor([[-0.0482, -0.0684,  0.1829,  ..., -0.2137,  0.1857,  0.1891],
        [ 0.0370, -0.0011,  0.1116,  ..., -0.1033,  0.0084,  0.0612],
        [ 0.0258,  0.0194,  0.1455,  ..., -0.1985,  0.0530,  0.0386],
        ...,
        [ 0.0802, -0.0974,  0.1981,  ..., -0.2107,  0.1324,  0.2148],
        [ 0.1956, -0.1600,  0.2822,  ..., -0.3600,  0.2164,  0.2478],
        [ 0.1956, -0.1600,  0.2822,  ..., -0.3600,  0.2164,  0.2478]])


In [68]:
# 2개 이상의 col을 선택할 때 텐서를 합쳐주는 부분입니다. 
# ex) ['자격요건','주요업무']를 합치는 경우 part 0,4를 합쳐주시면 됩니다.
combined_tensor = torch.cat([tensor_parts[0], tensor_parts[1], tensor_parts[4]], dim=1)

# combined_tensor 사이즈 확인
print(f"Combined Tensor Size: {combined_tensor.size()}")
print("Combined Tensor:")
print(combined_tensor)


Combined Tensor Size: torch.Size([12491, 2304])
Combined Tensor:
tensor([[-0.0482, -0.0684,  0.1829,  ..., -0.2828,  0.1753,  0.2709],
        [ 0.0370, -0.0011,  0.1116,  ..., -0.3179,  0.2235,  0.1840],
        [ 0.0258,  0.0194,  0.1455,  ..., -0.2688,  0.2132,  0.1755],
        ...,
        [ 0.0802, -0.0974,  0.1981,  ..., -0.3072,  0.1223,  0.2173],
        [ 0.1956, -0.1600,  0.2822,  ..., -0.3288,  0.1136,  0.1573],
        [ 0.1956, -0.1600,  0.2822,  ..., -0.3288,  0.1136,  0.1573]])


In [11]:
start_time = time.time()

# 임베딩 벡터에 url을 붙여줍니다.
new_embedding_vectors = vector_with_url(여기에 만든 one_tensor OR combined_tensor를 넣어주세요)


# DataFrame 생성 및 csv 파일 생성
# col_name = '주요업무' <- 실제 작업 시에는 이런 식으로 넣어주세요! 
col_name = '전체컬럼'

df = cos_similarity_df(new_embedding_vectors)
df.to_csv(f"cosine_cal_{col_name}.csv",index=False)

#시간 측정
end_time = time.time()
execution_time = end_time - start_time

hours = int(execution_time // 3600)
minutes = int((execution_time % 3600) // 60)
seconds = int(execution_time % 60)
print("작업 실행 시간: {}시간 {}분 {}초".format(hours, minutes, seconds))

# 병렬계산 방식 추가, 결과 리스트를 만들지 않고 확인하는 방법

In [ ]:
# 병렬계산

def cos_sim_cross(vec: torch.tensor):
    numerator = torch.mm(vec, vec.T)
    root_square_sum_vec = torch.sqrt(torch.sum(torch.square(vec), dim=1, keepdim=True))
    denominator = root_square_sum_vec * root_square_sum_vec.T
    return numerator / denominator

In [ ]:
# 결과 리스트 만드는건데 오래 걸려서 안씀

# def make_score_url_list(cos_sim: torch.tensor, url: list):
#     result = []
#     for row in tqdm(range(len(url))):
#         for col in range(row + 1, len(url)):
#             result.append((cos_sim[row, col].item(), url[row], url[col]))

#     return result

In [69]:
cos_sim = cos_sim_cross(combined_tensor)
url = pd.read_csv('../data/JobDescription/pre_result.csv')['출처 URL']

# 결과를 높은 순으로 정렬
cos_sim_argsort = cos_sim.argsort(descending=True, dim=1)

torch.Size([1, 12491])


In [70]:
# 0번 공고와 가장 유사한 것은 순서대로 0, 5987, 3242, ......

cos_sim_argsort

tensor([[    0,  3242,  5987,  ...,  5701,  5375, 11049],
        [    1,  1292,  2391,  ...,  1032,  5125,  4771],
        [ 1198,  1293,     2,  ...,  6680,  1655,   499],
        ...,
        [12488,  7978,  8453,  ...,  5701,  5375,  6253],
        [12490, 12489, 11492,  ..., 11327, 10818, 11049],
        [12489, 12490, 11492,  ..., 11327, 10818, 11049]])

In [ ]:
# 왜 2번 공고와 가장 비슷한 공고가 2번 말고도 1293, 1198 등이 있지?

print(url[2], url[1293], url[1198])

https://www.wanted.co.kr/wd/167134 https://www.wanted.co.kr/wd/167134 https://www.wanted.co.kr/wd/167134


In [78]:
jd_num = 0 # 0 to len(cos_sim_argsort) - 1
count = 10

print(f'기준 url: {url[jd_num]}')
print(f'가장 유사한 {count}개 공고 (동일 공고 제외)')

i = 0
while count > 0:
    obj = cos_sim_argsort[jd_num, i].item()
    if url[jd_num] == url[obj]:
        i += 1
        continue

    count -= 1
    print(url[obj], cos_sim[jd_num, obj])
    i += 1

기준 url: https://www.wanted.co.kr/wd/167436
가장 유사한 10개 공고 (동일 공고 제외)
https://www.jumpit.co.kr/position/17437 tensor(0.9733)
https://www.wanted.co.kr/wd/127697 tensor(0.9732)
https://www.wanted.co.kr/wd/127697 tensor(0.9732)
https://www.wanted.co.kr/wd/153609 tensor(0.9727)
https://www.jumpit.co.kr/position/13709 tensor(0.9725)
https://www.jumpit.co.kr/position/2035 tensor(0.9720)
https://www.wanted.co.kr/wd/101462 tensor(0.9717)
https://www.wanted.co.kr/wd/160630 tensor(0.9714)
https://www.wanted.co.kr/wd/160630 tensor(0.9714)
https://www.jumpit.co.kr/position/5028 tensor(0.9712)


In [79]:
jd_num = 0 # 0 to len(cos_sim_argsort) - 1
count = 10

print(f'기준 url: {url[jd_num]}')
print(f'가장 유사하지 않은 {count}개 공고 (동일 공고 제외)')

i = len(cos_sim_argsort) - 1
while count > 0:
    obj = cos_sim_argsort[jd_num, i].item()
    if url[jd_num] == url[obj]:
        i -= 1
        continue

    count -= 1
    print(url[obj], cos_sim[jd_num, obj])
    i -= 1

기준 url: https://www.wanted.co.kr/wd/167436
가장 유사하지 않은 10개 공고 (동일 공고 제외)
https://www.jumpit.co.kr/position/16294 tensor(0.0944)
https://www.wanted.co.kr/wd/24568 tensor(0.1378)
https://www.wanted.co.kr/wd/24568 tensor(0.1378)
https://www.wanted.co.kr/wd/24568 tensor(0.1378)
https://www.wanted.co.kr/wd/117393 tensor(0.1730)
https://www.wanted.co.kr/wd/117393 tensor(0.1730)
https://www.wanted.co.kr/wd/130770 tensor(0.1816)
https://www.wanted.co.kr/wd/130770 tensor(0.1816)
https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1245149 tensor(0.2159)
https://www.wanted.co.kr/wd/164828 tensor(0.2165)
